In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
from jaqs_fxdayu.data.dataservice import LocalDataService
ds = LocalDataService()
from time import time
## 加freq参数


In [2]:
#通过测试报告，看多空信号的绩效，这里是定义报告格式
from jaqs_fxdayu.research import TimingDigger

#td = TimingDigger(output_folder=".", output_format='pdf',signal_name='alpha60')

def TimingSignal(td, dv, long='Long', short='Short',closeLong='closeLong', closeShort='closeShort'\
                 , mhp=None,sl=None,sp=None):
    # step 1：实例化TimingDigger 通过output_folder和output_format指定测试报告的输出路径和输出格式，通过signal_name指定绩效文件名称

    #多空信号分别计算一遍 输出汇总结果
    td.process_signal(
        enter_signal=dv.get_ts(long),
        exit_signal=dv.get_ts(closeLong),
        sig_type="long",  # 信号类型 long/short
        price=dv.get_ts('close'),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )

    td.process_signal(
        enter_signal=dv.get_ts(short),
        exit_signal=dv.get_ts(closeShort),
        sig_type="short",  # 信号类型 long/short
        price=dv.get_ts("close"),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )


F:\Anaconda\envs\dayu\lib\site-packages\jaqs\research\signaldigger\plotting.py:10: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  mpl.use('Agg')
F:\Anaconda\envs\dayu\lib\site-packages\matplotlib\__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [3]:
start = time()
path = r'./min_data/VnTrader_1Min_Db'
props = {'fields': 'open,high,low,close,volume','symbol': 'BTCUSDT:binance', 'freq': '1D',
         'start_date':20180101000000}

Time_dict = ds.bar_reader(path,props)
print(time()-start)

8.598069906234741


In [4]:
from jaqs_fxdayu.data.hf_dataview import HFDataView
dv = HFDataView()

In [5]:
dv.create_init_dv(Time_dict.set_index(["trade_date","symbol"]))
data1D = Time_dict.set_index('datetime')

Initialize dataview success.


价量趋势(Price and Volume Trend)指标．把能量变化与价格趋势有机地联系到了一起，从而构成了量价趋势指标

In [6]:
PVT = dv.add_formula('PVT','Return(close,1) * volume * 0.000001',add_data=True)
print(float(PVT.max()-(PVT.max()-PVT.min())*0.4))

0.0030860768796606465


In [ ]:
入场信号：当PVT值大于 PVT.max()-(PVT.max()-PVT.min())*0.4，买入，近似于选择排名前60%的数据
出场信号：当PVT值小于 PVT.max()-(PVT.max()-PVT.min())*0.4，卖入，当指标低于这个值的时候说明信号并不强烈，可以卖出

In [7]:
Long = dv.add_formula('Long','If(PVT>0.0030860768796606465, 2, 0)', add_data=True)
Short = dv.add_formula('Short','If(PVT<0.0030860768796606465, -2, 0)', add_data=True)

close_long = dv.add_formula("closeLong","If(Short==-2,1,0)", add_data=True)
close_short = dv.add_formula("closeShort","If(Long==2,-1,0)", add_data=True)

In [8]:
td = TimingDigger(output_folder=".", output_format='pdf',signal_name='PVT')
TimingSignal(td, dv, long='Long', short='Short',closeLong='closeLong', closeShort='closeShort',mhp=100, sl=0.05, sp=0.3)#调取数据

Nan Data Count (should be zero) : 0;  Percentage of effective data: 5%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 88%


In [9]:
td.create_event_report(sig_type="long")

*****-Summary-*****
Event Analysis
                       win   loss     all
t-stat               3.992 -1.422   1.792
p-value              0.003  0.291   0.098
mean                 0.039 -0.033   0.022
std                  0.029  0.033   0.043
info_ratio           1.331 -1.005   0.517
skewness             0.716 -0.685  -0.481
kurtosis            -0.661 -1.500   0.558
pct5                 0.010 -0.073  -0.040
pct25                0.012 -0.047   0.009
pct50                0.031 -0.013   0.017
pct75                0.058 -0.010   0.049
pct95                0.086 -0.007   0.082
occurance           10.000  3.000  13.000
win_ratio              NaN    NaN   0.769
win_mean/loss_mean     NaN    NaN   1.163


In [10]:
td.create_event_report(sig_type="short")

*****-Summary-*****
Event Analysis
                        win     loss      all
t-stat               16.053  -24.716    4.773
p-value               0.000    0.000    0.000
mean                  0.185   -0.075    0.049
std                   0.121    0.033    0.157
info_ratio            1.531   -2.247    0.313
skewness              0.171   -1.761    0.772
kurtosis             -1.151    7.946   -0.641
pct5                  0.007   -0.129   -0.121
pct25                 0.081   -0.092   -0.067
pct50                 0.179   -0.066   -0.028
pct75                 0.310   -0.057    0.174
pct95                 0.357   -0.031    0.339
occurance           111.000  122.000  233.000
win_ratio               NaN      NaN    0.476
win_mean/loss_mean      NaN      NaN    2.471


In [11]:
td.create_event_report(sig_type="long_short") #同时计算多空头信号的绩效

*****-Summary-*****
Event Analysis
                        win     loss      all
t-stat               15.417  -24.252    4.880
p-value               0.000    0.000    0.000
mean                  0.173   -0.074    0.048
std                   0.123    0.034    0.153
info_ratio            1.407   -2.178    0.312
skewness              0.290   -1.605    0.812
kurtosis             -1.153    7.352   -0.505
pct5                  0.007   -0.129   -0.121
pct25                 0.062   -0.091   -0.066
pct50                 0.168   -0.066   -0.007
pct75                 0.305   -0.057    0.166
pct95                 0.355   -0.021    0.338
occurance           121.000  125.000  246.000
win_ratio               NaN      NaN    0.492
win_mean/loss_mean      NaN      NaN    2.341


可以看出输出的绩效报告之中win_ratio>0.4,win_mean/loss_mean>1，整体来说效果不错

In [12]:
start = time()
path = r'./min_data/VnTrader_1Min_Db'
props = {'fields': 'open,high,low,close,volume','symbol': 'EOSUSDT:binance', 'freq': '1D',
         'start_date':20180101000000}

Time_dict = ds.bar_reader(path,props)
print(time()-start)

4.481421232223511


In [13]:
dv.create_init_dv(Time_dict.set_index(["trade_date","symbol"]))
data1D = Time_dict.set_index('datetime')

Initialize dataview success.


In [14]:
PVT = dv.add_formula('PVT','Return(close,1) * volume * 0.000001',add_data=True)
print(float(PVT.max()-(PVT.max()-PVT.min())*0.4))

0.29527080494385616


In [15]:
Long = dv.add_formula('Long','If(PVT>0.29527080494385616, 2, 0)', add_data=True)
Short = dv.add_formula('Short','If(PVT<0.29527080494385616, -2, 0)', add_data=True)

close_long = dv.add_formula("closeLong","If(Short==-2,1,0)", add_data=True)
close_short = dv.add_formula("closeShort","If(Long==2,-1,0)", add_data=True)

In [16]:
td = TimingDigger(output_folder=".", output_format='pdf',signal_name='PVT1')
TimingSignal(td, dv, long='Long', short='Short',closeLong='closeLong', closeShort='closeShort',mhp=100, sl=0.05, sp=0.3)#调取数据

Nan Data Count (should be zero) : 0;  Percentage of effective data: 23%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 74%


In [17]:
td.create_event_report(sig_type="long")

*****-Summary-*****
Event Analysis
                       win    loss     all
t-stat               4.699  -4.062   0.552
p-value              0.000   0.003   0.586
mean                 0.050  -0.065   0.008
std                  0.043   0.048   0.072
info_ratio           1.175  -1.354   0.108
skewness             0.863  -0.378  -0.278
kurtosis            -0.483  -0.993  -0.203
pct5                 0.003  -0.140  -0.117
pct25                0.022  -0.098  -0.033
pct50                0.037  -0.058   0.014
pct75                0.070  -0.032   0.040
pct95                0.127  -0.004   0.120
occurance           17.000  10.000  27.000
win_ratio              NaN     NaN   0.630
win_mean/loss_mean     NaN     NaN   0.778


In [18]:
td.create_event_report(sig_type="short")

*****-Summary-*****
Event Analysis
                       win    loss     all
t-stat               8.583  -6.442   3.572
p-value              0.000   0.000   0.001
mean                 0.170  -0.059   0.059
std                  0.132   0.059   0.154
info_ratio           1.294  -1.006   0.385
skewness             0.066  -1.453   0.602
kurtosis            -1.750   1.357  -0.707
pct5                 0.012  -0.197  -0.146
pct25                0.038  -0.083  -0.038
pct50                0.155  -0.040   0.008
pct75                0.309  -0.017   0.157
pct95                0.327  -0.001   0.323
occurance           45.000  42.000  87.000
win_ratio              NaN     NaN   0.517
win_mean/loss_mean     NaN     NaN   2.869


In [19]:
td.create_event_report(sig_type="long_short")

*****-Summary-*****
Event Analysis
                       win    loss      all
t-stat               8.503  -7.560    3.559
p-value              0.000   0.000    0.001
mean                 0.138  -0.060    0.047
std                  0.126   0.057    0.141
info_ratio           1.089  -1.059    0.335
skewness             0.537  -1.323    0.777
kurtosis            -1.412   1.132   -0.152
pct5                 0.007  -0.190   -0.133
pct25                0.026  -0.084   -0.037
pct50                0.080  -0.041    0.010
pct75                0.294  -0.020    0.095
pct95                0.324  -0.001    0.320
occurance           62.000  52.000  114.000
win_ratio              NaN     NaN    0.544
win_mean/loss_mean     NaN     NaN    2.275


在新的品种中同样的进出场信号，可以发现win_ratio都是大于0.5的，除了long的绩效报告中win_mean/loss_mean<1之外，其他的win_mean/loss_mean都大于2，除了纯多头的效果不好之外，其他的效果都比较显著